In [ ]:
# datda cleaning
import os
import pandas as pd
from datetime import datetime

# Filepath for the input and output files
home_dir = os.path.expanduser('~')
input_file = f"{home_dir}/nfs/data/amd_5y_bar.csv"
output_file = f"{home_dir}/chan/amd_day.csv"

df = pd.read_csv(input_file)
df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0", "volume", "symbol"], errors="ignore")
df['timestamp'] = pd.to_datetime(df['timestamp'] // 1_000_000, unit='s')
df.columns = [col.lower() for col in df.columns]
df.to_csv(output_file, index=False)

print(f"Data cleaned and saved to {output_file}")


In [ ]:
from Chan import CChan
from ChanConfig import CChanConfig
from Common.CEnum import AUTYPE, DATA_SRC, KL_TYPE
from DataAPI.csvAPI import CSV_API  # Import CSV_API from its module
from Plot.AnimatePlotDriver import CAnimateDriver
from Plot.PlotDriver import CPlotDriver

if __name__ == "__main__":
    code = "amd"
    begin_time = "2023-01-01"
    end_time = None
    data_src = DATA_SRC.CSV
    lv_list = [KL_TYPE.K_DAY]

    config = CChanConfig({
        "bi_strict": True,
        "trigger_step": True,
        "skip_step": 0,
        "divergence_rate": float("inf"),
        "bsp2_follow_1": False,
        "bsp3_follow_1": False,
        "min_zs_cnt": 0,
        "bs1_peak": False,
        "macd_algo": "peak",
        "bs_type": '1,2,3a,1p,2s,3b',
        "print_warning": True,
        "zs_algo": "normal",
    })

    plot_config = {
        "plot_kline": True,
        "plot_kline_combine": True,
        "plot_bi": True,
        "plot_seg": True,
        "plot_eigen": False,
        "plot_zs": True,
        "plot_macd": False,
        "plot_mean": False,
        "plot_channel": False,
        "plot_bsp": True,
        "plot_extrainfo": False,
        "plot_demark": False,
        "plot_marker": False,
        "plot_rsi": False,
        "plot_kdj": False,
    }

    plot_para = {
        "seg": {
            # "plot_trendline": True,
        },
        "bi": {
            # "show_num": True,
            # "disp_end": True,
        },
        "figure": {
            "x_range": 200,
        },
        "marker": {
            # "markers": {  # text, position, color
            #     '2023/06/01': ('marker here', 'up', 'red'),
            #     '2023/06/08': ('marker here', 'down')
            # },
        }
    }
    csv_api = CSV_API(
        code=code,
        k_type=KL_TYPE.K_DAY,
        begin_date=begin_time,
        end_date=end_time
    )


    chan = CChan(
        code=code,
        begin_time=begin_time,
        end_time=end_time,
        data_src=data_src,
        lv_list=lv_list,
        config=config,
        autype=AUTYPE.QFQ,
    )

    if not config.trigger_step:
        plot_driver = CPlotDriver(
            chan,
            plot_config=plot_config,
            plot_para=plot_para,
        )
        plot_driver.figure.show()
        plot_driver.save2img("./test.png")
    else:
        CAnimateDriver(
            chan,
            plot_config=plot_config,
            plot_para=plot_para,
        )
